<a href="https://colab.research.google.com/github/Anyajain-Tech/AI-Project/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math           # import to use the built in function squareroot(sqrt) , logathrim (log)
import torch          #PyTorch  package
import torch.nn as nn     # it is import to use neural-network layer like Linear , LayerNorm
import matplotlib.pyplot as plt

#from transformers import AutoTokenizer , AutoModelForCausalLM

In [ ]:
class PositionalEncoding  (nn.Module):        #defining a positional encoding class a child class calling  parent PyTorch class for Neural network
  def __init__ (self , d_module: int , max_len : int =5000):    #creating constructor giving argument d_module , max_lex and giving dataType bascially defining
                                                                 # how this constructor is be created
    """documentation string

    """

    super().__init__()                  #calling nn.module constructor
    position = torch.arange(0, max_len , dtype=torch.float32).unsqueeze(1)    # from torch arange creating [0,1,2,3....,max_len-1] through unsequeeze adding new dimension
    divisions=torch.exp(torch.arange(0, d_module , 2, dtype=torch.float32))*(-math.log(10000) / d_module)

    arr= torch.zeros(max_len ,d_module)
    arr[:,0::2] = torch.sin(position*divisions)
    arr[:,1::2] = torch.cos(position*divisions)
    arr=arr.unsequeeze(0)

    self.register_buffer("arr" , arr)
def forward(self ,x: torch.Tensor)->torch.Tensor:
  """documentation string
    x:[batch,seq_len,d_model]
  """
  seq_len =x.size(1)
  p=x+self.arr[:,:seq_len , :]
  return p


In [ ]:
class MultiHeadSelfAttention (nn.Module):
  def __init__(self , d_model:int , num_heads: int , dropout : float=0.1):
    """

    """
    super().__init__()
    assert d_model % num_heads ==0
    self.d_model =d_model
    self.num = num_heads
    self.num_heads = num_heads
    self.head_dim = d_model // num_heads

    self.q_proj = nn.Linear(d_model , d_model)
    self.k_proj = nn.Linear (d_model ,d_model)
    self.v_proj = nn.Linear (d_model ,d_model)

    self.out_proj = nn.Linear (d_model , d_model)
    self.dropout= nn.Dropout(dropout)


  def forward(self, x:torch.Tensor)->torch.Tensor:
    """

    """
    B,T,C =x.size()
    Q =self.q_proj(x)
    K =self.k_proj(x)
    V=self.v_proj(x)

    Q=Q.view(B,T,self.num_heads ,self.head_dim).T(1,2)
    K=K.view(B,T,self.num_heads, self.head_dim).T(1,2)
    V=V.view(B,T,self.num_heads, self.head_dim).T(1,2)

    scores= (Q @ K.T(-2,-1)) / math.sqrt(self.head_dim)
    mask =torch.triu(torch.ones(T,T,device=x.device) , diagonal=1).bool()
    scores = scores.masked_fill(mask,float("-inf"))                   #-infinity
    attention=self.dropout(torch.softmax(scores ,dim=-1))
    out= attention@V
    out=out.T(1,2).contiguous().view(B,T,c)
    out = self.out_proj(out)
    return out


In [ ]:
class GPT2 (nn.Module):
  def __init__(self, d_model : int , num_heads : int , d_ff : int , dropout:float=0.1):
    """

    """
    super().__init__()
    self.layer1 = nn.LayerNorm(d_model)
    self.attention = MultiHeadSelfAttention(d_model,num_heads,dropout)
    self.layer2 =nn.LayerNorm(d_model)
    self.ff = nn.sequential(nn.Linear(d_model ,d_ff ) , nn.GELU() , nn.Linear(d_ff ,d_model) ,nn.Dropout(dropout) )


  def forward (self , x: torch.Tensor) ->torch.Tensor :
    """

    """
    x+=self.attention (self.layer1(x))          #attention +residual left
    x+=self.ff(self.layer2(x))
    return x